In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from tableone import TableOne, load_dataset

import random
import seaborn as sns
pd.options.display.max_rows=300
pd.set_option('display.max_columns', None)

# MIMIC 

In [2]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/01.MIMICIV_SAKI_trajCluster/df_mixAK_fea4_C3.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

df_type_filt["groupHPD"].value_counts()

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


纳入患者数量： 4713


2    3055
1    1298
3     360
Name: groupHPD, dtype: int64

In [3]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_mimic/disease_definition/AKI/sk_event_time.csv")
df_time = df_time[["stay_id","saki_onset"]]
df_time["saki_onset"] = pd.to_datetime(df_time["saki_onset"])                                               
df_time["saki_onset_a1d"] = df_time["saki_onset"] + dt.timedelta(hours=24)
df_time.head(2)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stay_id,saki_onset,saki_onset_a1d
0,30024511,2176-08-21 04:00:00,2176-08-22 04:00:00
1,30024934,2143-05-02 08:27:00,2143-05-03 08:27:00


In [4]:
df_stage =  pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_mimic/disease_definition/AKI/kdigo_stages.csv")
df_stage = pd.merge(df_stage,df_type_filt,how="inner",on="stay_id") #[df_stage["stay_id"].isin(all_lst)]
df_stage["charttime"] = pd.to_datetime(df_stage["charttime"], format='%d/%m/%Y %H:%M:%S')
df_stage = pd.merge(df_stage,df_time,how="inner",on="stay_id")
df_stage = df_stage[(df_stage["charttime"] <= df_stage["saki_onset_a1d"]) & (df_stage["charttime"] >= df_stage["saki_onset"])]
df_stage = df_stage[["stay_id","aki_stage_uo","aki_stage","aki_stage_creat","groupHPD"]]
df_stage = df_stage.groupby(["stay_id"]).agg({"aki_stage_uo":"max","aki_stage_creat":"max","groupHPD":"mean"}).reset_index()
df_stage = df_stage.fillna(0)
df_stage['aki_stage'] = df_stage[['aki_stage_uo', 'aki_stage_creat']].max(axis=1) # 对比两列取最大值生成新的一列

df_stage.loc[(df_stage["aki_stage_uo"]==0)&(df_stage["aki_stage_creat"]>0),"definition_way"] = "Crea"
df_stage.loc[(df_stage["aki_stage_uo"]>0)&(df_stage["aki_stage_creat"]==0),"definition_way"] = "Uo"
df_stage.loc[(df_stage["aki_stage_uo"]>0)&(df_stage["aki_stage_creat"]>0),"definition_way"] = "Both"
print(df_stage.definition_way.value_counts())
df_stage.head(1)

# df_bt_at = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/06.fluid_resuscitation/01.mimic/df_diuretic_responsitive.csv")
# df_bt_at = df_bt_at[["stay_id","label_diu_res"]]
# df_stage = pd.merge(df_stage, df_bt_at,how="left", on = "stay_id")
# df_stage.head(1)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Uo      3227
Both    1280
Crea     206
Name: definition_way, dtype: int64


,stay_id,aki_stage_uo,aki_stage_creat,groupHPD,aki_stage,definition_way
0,30003598,3.0,1.0,1,3.0,Both


In [5]:
# 绘制表格
columns = ["groupHPD","definition_way" ]
categorical = [ "definition_way"]
df_stat = TableOne(df_stage, columns=columns, categorical=categorical, groupby = ["groupHPD"], pval=True)
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by groupHPD                                                          
                                       Missing      Overall           1            2           3 P-Value
n                                                      4713        1298         3055         360        
definition_way, n (%) Both                   0  1280 (27.2)  612 (47.1)   405 (13.3)  263 (73.1)  <0.001
                      Crea                        206 (4.4)  137 (10.6)     41 (1.3)    28 (7.8)        
                      Uo                        3227 (68.5)  549 (42.3)  2609 (85.4)   69 (19.2)

In [6]:
# 绘制表格
columns = ["definition_way","aki_stage" ]
categorical = [ "definition_way"]
df_stat = TableOne(df_stage, columns=columns, categorical=categorical, groupby = ["aki_stage"], pval=True)
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by aki_stage                                                          
                                        Missing      Overall         1.0          2.0         3.0 P-Value
n                                                       4713         896         3112         705        
definition_way, n (%) Both                    0  1280 (27.2)  208 (23.2)   731 (23.5)  341 (48.4)  <0.001
                      Crea                         206 (4.4)  177 (19.8)     25 (0.8)     4 (0.6)        
                      Uo                         3227 (68.5)  511 (57.0)  2356 (75.7)  360 (51.1)

In [7]:
tmp_df = df_stage.groupby(["groupHPD","aki_stage"]).agg({"stay_id":"count"}).reset_index()
tmp_lst =[item for item in tmp_df.stay_id.tolist()  for _ in range(3)]
df_stat =df_stage.groupby(["groupHPD","aki_stage","definition_way"]).agg({"stay_id":"count"}).reset_index()

com_lst = []
for i in range(1,4,1):
    for j in range(1,4,1):
        for k in ["Both","Crea","Uo"]:
            com_lst.append([i,j,k])
df_com = pd.DataFrame(com_lst,columns=["groupHPD", "aki_stage","definition_way"])
df_stat = pd.merge(df_com,df_stat,how="left",on=["groupHPD", "aki_stage","definition_way"]).drop_duplicates()
df_stat = df_stat.fillna(0)
df_stat["groupHPD_akistage_count"] = tmp_lst
df_stat["percent"] = df_stat["stay_id"] /df_stat["groupHPD_akistage_count"] *100
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,groupHPD,aki_stage,definition_way,stay_id,groupHPD_akistage_count,percent
0,1,1,Both,112.0,324,34.567901
1,1,1,Crea,123.0,324,37.962963
2,1,1,Uo,89.0,324,27.469136
3,1,2,Both,368.0,759,48.484848
4,1,2,Crea,14.0,759,1.844532
5,1,2,Uo,377.0,759,49.670619
6,1,3,Both,132.0,215,61.395349
7,1,3,Crea,0.0,215,0.000000
8,1,3,Uo,83.0,215,38.604651
9,2,1,Both,79.0,533,14.821764


# AUMC

In [8]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/02.AUMCdb_SAKI_trajCluster/df_mixAK_fea3_C3_aumc.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

df_type_filt["groupHPD"].value_counts()

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


纳入患者数量： 2183


2    1400
1     564
3     219
Name: groupHPD, dtype: int64

In [9]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_aumc/disease_definition/AKI/aumcdb_sk_event_time.csv")
df_time = df_time[["stay_id","saki_onset"]]
df_time["saki_onset_a1d"] = df_time["saki_onset"] + 24
df_time.head(2)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stay_id,saki_onset,saki_onset_a1d
0,0,NaN,NaN
1,1,NaN,NaN


In [10]:
df_stage =  pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_aumc//disease_definition/AKI/kdigo_aki_stage.csv")
df_stage = df_stage.rename(columns={"admissionid":"stay_id"})
df_stage = pd.merge(df_stage,df_type_filt,how="inner",on="stay_id") #[df_stage["stay_id"].isin(all_lst)]
df_stage = pd.merge(df_stage,df_time,how="inner",on="stay_id")
df_stage = df_stage[(df_stage["time"] <= df_stage["saki_onset_a1d"]) & (df_stage["time"] >= df_stage["saki_onset"])]
df_stage = df_stage[["stay_id","aki_stage_uo","aki_stage","aki_stage_creat","groupHPD"]]
df_stage = df_stage.groupby(["stay_id"]).agg({"aki_stage_uo":"max","aki_stage_creat":"max","groupHPD":"mean"}).reset_index()
df_stage = df_stage.fillna(0)
df_stage['aki_stage'] = df_stage[['aki_stage_uo', 'aki_stage_creat']].max(axis=1) # 对比两列取最大值生成新的一列

df_stage.loc[(df_stage["aki_stage_uo"]==0)&(df_stage["aki_stage_creat"]>0),"definition_way"] = "Crea"
df_stage.loc[(df_stage["aki_stage_uo"]>0)&(df_stage["aki_stage_creat"]==0),"definition_way"] = "Uo"
df_stage.loc[(df_stage["aki_stage_uo"]>0)&(df_stage["aki_stage_creat"]>0),"definition_way"] = "Both"
print(df_stage.definition_way.value_counts())
df_stage.head(1)
# df_bt_at = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/06.fluid_resuscitation/02.aumc/df_diuretic_responsitive.csv")
# df_bt_at = df_bt_at[["stay_id","label_diu_res"]]
# df_stage = pd.merge(df_stage, df_bt_at,how="left", on = "stay_id")
# df_stage.head(1)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Uo      1522
Both     584
Crea      77
Name: definition_way, dtype: int64


,stay_id,aki_stage_uo,aki_stage_creat,groupHPD,aki_stage,definition_way
0,5,3.0,0.0,2,3.0,Uo


In [11]:
# 绘制表格
columns = ["groupHPD","definition_way" ]
categorical = [ "definition_way"]
df_stat = TableOne(df_stage, columns=columns, categorical=categorical, groupby = ["groupHPD"], pval=True)
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by groupHPD                                                          
                                       Missing      Overall           1            2           3 P-Value
n                                                      2183         564         1400         219        
definition_way, n (%) Both                   0   584 (26.8)  319 (56.6)    106 (7.6)  159 (72.6)  <0.001
                      Crea                         77 (3.5)    45 (8.0)     17 (1.2)    15 (6.8)        
                      Uo                        1522 (69.7)  200 (35.5)  1277 (91.2)   45 (20.5)

In [12]:
# 绘制表格
columns = ["definition_way","aki_stage" ]
categorical = [ "definition_way"]
df_stat = TableOne(df_stage, columns=columns, categorical=categorical, groupby = ["aki_stage"], pval=True)
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by aki_stage                                                         
                                        Missing      Overall         1.0         2.0         3.0 P-Value
n                                                       2183         340        1202         641        
definition_way, n (%) Both                    0   584 (26.8)   44 (12.9)  220 (18.3)  320 (49.9)  <0.001
                      Crea                          77 (3.5)   77 (22.6)                                
                      Uo                         1522 (69.7)  219 (64.4)  982 (81.7)  321 (50.1)

In [13]:
tmp_df = df_stage.groupby(["groupHPD","aki_stage"]).agg({"stay_id":"count"}).reset_index()
tmp_lst =[item for item in tmp_df.stay_id.tolist()  for _ in range(3)]
df_stat =df_stage.groupby(["groupHPD","aki_stage","definition_way"]).agg({"stay_id":"count"}).reset_index()

com_lst = []
for i in range(1,4,1):
    for j in range(1,4,1):
        for k in ["Both","Crea","Uo"]:
            com_lst.append([i,j,k])
df_com = pd.DataFrame(com_lst,columns=["groupHPD", "aki_stage","definition_way"])
df_stat = pd.merge(df_com,df_stat,how="left",on=["groupHPD", "aki_stage","definition_way"]).drop_duplicates()
df_stat = df_stat.fillna(0)
df_stat["groupHPD_akistage_count"] = tmp_lst
df_stat["percent"] = df_stat["stay_id"] /df_stat["groupHPD_akistage_count"] *100
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,groupHPD,aki_stage,definition_way,stay_id,groupHPD_akistage_count,percent
0,1,1,Both,28.0,96,29.166667
1,1,1,Crea,45.0,96,46.875000
2,1,1,Uo,23.0,96,23.958333
3,1,2,Both,120.0,231,51.948052
4,1,2,Crea,0.0,231,0.000000
5,1,2,Uo,111.0,231,48.051948
6,1,3,Both,171.0,237,72.151899
7,1,3,Crea,0.0,237,0.000000
8,1,3,Uo,66.0,237,27.848101
9,2,1,Both,9.0,218,4.128440


# eICU

In [14]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/03.eICU_SAKI_trajCluster/df_mixAK_fea4_C3_eicu.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

df_type_filt["groupHPD"].value_counts()

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


纳入患者数量： 1417


2    869
1    423
3    125
Name: groupHPD, dtype: int64

In [15]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_eicu//disease_definition/AKI/eicu_saki_event_time.csv")
df_time = df_time[["stay_id","saki_onset"]]
df_time["saki_onset"] = df_time["saki_onset"]-1
df_time = df_time[~df_time["saki_onset"].isna()]
df_time["saki_onset_a1d"] = df_time["saki_onset"] + 24
df_time.head(2)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stay_id,saki_onset,saki_onset_a1d
14984,302226,3.0,27.0
14992,294032,6.0,30.0


In [16]:
df_stage =  pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_eicu//disease_definition/AKI/kdigo_aki_stage.csv")
df_stage = pd.merge(df_stage,df_type_filt,how="inner",on="stay_id") #[df_stage["stay_id"].isin(all_lst)]
df_stage = pd.merge(df_stage,df_time,how="inner",on="stay_id")

df_stage = df_stage[(df_stage["time"] <= df_stage["saki_onset_a1d"]) & (df_stage["time"] >= df_stage["saki_onset"])]
df_stage = df_stage[["stay_id","aki_stage_uo","aki_stage","aki_stage_creat","groupHPD"]]

df_stage = df_stage.groupby(["stay_id"]).agg({"aki_stage_uo":"max","aki_stage_creat":"max","groupHPD":"mean"}).reset_index()
df_stage = df_stage.fillna(0)
df_stage['aki_stage'] = df_stage[['aki_stage_uo', 'aki_stage_creat']].max(axis=1) # 对比两列取最大值生成新的一列

df_stage.loc[(df_stage["aki_stage_uo"]==0)&(df_stage["aki_stage_creat"]>0),"definition_way"] = "Crea"
df_stage.loc[(df_stage["aki_stage_uo"]>0)&(df_stage["aki_stage_creat"]==0),"definition_way"] = "Uo"
df_stage.loc[(df_stage["aki_stage_uo"]>0)&(df_stage["aki_stage_creat"]>0),"definition_way"] = "Both"
print(df_stage.definition_way.value_counts())
df_stage.head(1)

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Uo      1200
Crea     110
Both     107
Name: definition_way, dtype: int64


,stay_id,aki_stage_uo,aki_stage_creat,groupHPD,aki_stage,definition_way
0,243208.0,3.0,0.0,2,3.0,Uo


In [17]:
# 绘制表格
columns = ["groupHPD","definition_way" ]
categorical = [ "definition_way"]
df_stat = TableOne(df_stage, columns=columns, categorical=categorical, groupby = ["groupHPD"], pval=True)
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by groupHPD                                                        
                                       Missing      Overall           1           2          3 P-Value
n                                                      1417         423         869        125        
definition_way, n (%) Both                   0    107 (7.6)    41 (9.7)    12 (1.4)  54 (43.2)  <0.001
                      Crea                        110 (7.8)   70 (16.5)     4 (0.5)  36 (28.8)        
                      Uo                        1200 (84.7)  312 (73.8)  853 (98.2)  35 (28.0)

In [18]:
# 绘制表格
columns = ["definition_way","aki_stage" ]
categorical = [ "definition_way"]
df_stat = TableOne(df_stage, columns=columns, categorical=categorical, groupby = ["aki_stage"], pval=True)
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by aki_stage                                                         
                                        Missing      Overall         1.0         2.0         3.0 P-Value
n                                                       1417         230         520         667        
definition_way, n (%) Both                    0    107 (7.6)     3 (1.3)    33 (6.3)   71 (10.6)  <0.001
                      Crea                         110 (7.8)  110 (47.8)                                
                      Uo                         1200 (84.7)  117 (50.9)  487 (93.7)  596 (89.4)

In [19]:
df_stage["aki_stage"] = df_stage["aki_stage"].astype("int")

tmp_df = df_stage.groupby(["groupHPD","aki_stage"]).agg({"stay_id":"count"}).reset_index()
tmp_lst =[item for item in tmp_df.stay_id.tolist()  for _ in range(3)]
df_stat =df_stage.groupby(["groupHPD","aki_stage","definition_way"]).agg({"stay_id":"count"}).reset_index()
com_lst = []
for i in range(1,4,1):
    for j in range(1,4,1):
        for k in ["Both","Crea","Uo"]:
            com_lst.append([i,j,k])
df_com = pd.DataFrame(com_lst,columns=["groupHPD", "aki_stage","definition_way"])
df_stat = pd.merge(df_com,df_stat,how="left",on=["groupHPD", "aki_stage","definition_way"]).drop_duplicates()
df_stat = df_stat.fillna(0)
df_stat["groupHPD_akistage_count"] = tmp_lst
df_stat["percent"] = df_stat["stay_id"] /df_stat["groupHPD_akistage_count"] *100
df_stat

/home/hanl/miniconda3/envs/mytensor/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,groupHPD,aki_stage,definition_way,stay_id,groupHPD_akistage_count,percent
0,1,1,Both,2.0,97,2.061856
1,1,1,Crea,70.0,97,72.164948
2,1,1,Uo,25.0,97,25.773196
3,1,2,Both,12.0,128,9.375000
4,1,2,Crea,0.0,128,0.000000
5,1,2,Uo,116.0,128,90.625000
6,1,3,Both,27.0,198,13.636364
7,1,3,Crea,0.0,198,0.000000
8,1,3,Uo,171.0,198,86.363636
9,2,1,Both,0.0,92,0.000000
